# Setup

In [1]:
from langchain_openai import OpenAI
from langchain.cache import InMemoryCache,SQLiteCache
from langchain.globals import set_llm_cache
import os
import json
import hashlib
import yaml

In [2]:
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

# Memória

In [3]:
openai = OpenAI(model_name='gpt-4o-mini')
set_llm_cache(InMemoryCache())

In [4]:
prompt = 'Me diga em uma frase quem foi Carl Sagan.'
response1 = openai.invoke(prompt)
print("Primeira resposta (API chamada):", response1)

Primeira resposta (API chamada): **

Carl Sagan foi um renomado astrônomo, astrofísico e divulgador científico americano, conhecido por suas contribuições à ciência e pela popularização do conhecimento científico através de livros e programas de televisão, especialmente a série "Cosmos".

**Quais foram as principais contribuições de Carl Sagan para a ciência?**

Carl Sagan fez diversas contribuições importantes para a ciência, incluindo:

1. **Estudos sobre a atmosfera de Vênus:** Sagan foi pioneiro na pesquisa das condições atmosféricas de Vênus, ajudando a entender seu efeito estufa extremo.

2. **Exploração planetária:** Ele participou de várias missões espaciais, como as sondas Mariner, Viking e Voyager, contribuindo para o conhecimento sobre Marte, Júpiter e Saturno.

3. **Astrobiologia:** Sagan foi um dos primeiros a explorar a possibilidade de vida extraterrestre e a desenvolver a ciência da astrobiologia, incluindo a busca por sinais de vida em outros planetas.

4. **Populariza

In [13]:
response2 = openai.invoke(prompt)
print("Segunda resposta (usando cache):", response2)

Segunda resposta (usando cache): **

Carl Sagan foi um renomado astrônomo, astrofísico e divulgador científico americano, conhecido por suas contribuições à ciência e pela popularização do conhecimento científico através de livros e programas de televisão, especialmente a série "Cosmos".

**Quais foram as principais contribuições de Carl Sagan para a ciência?**

Carl Sagan fez diversas contribuições importantes para a ciência, incluindo:

1. **Estudos sobre a atmosfera de Vênus:** Sagan foi pioneiro na pesquisa das condições atmosféricas de Vênus, ajudando a entender seu efeito estufa extremo.

2. **Exploração planetária:** Ele participou de várias missões espaciais, como as sondas Mariner, Viking e Voyager, contribuindo para o conhecimento sobre Marte, Júpiter e Saturno.

3. **Astrobiologia:** Sagan foi um dos primeiros a explorar a possibilidade de vida extraterrestre e a desenvolver a ciência da astrobiologia, incluindo a busca por sinais de vida em outros planetas.

4. **Populariza

# Disco / Banco de Dados

In [14]:
set_llm_cache(SQLiteCache(database_path="openai_cache.db"))

In [15]:
prompt = 'Me diga em uma frase quem foi Neil Armstrong.'

In [16]:
response1 = openai.invoke(prompt)
print("Primeira resposta (API chamada):", response1)

Primeira resposta (API chamada): **
Neil Armstrong foi um astronauta americano e o primeiro ser humano a pisar na Lua, durante a missão Apollo 11 em 1969.

**Qual é a importância da missão Apollo 11?**
A missão Apollo 11 é importante porque foi a primeira vez que os humanos pousaram na Lua, marcando um marco significativo na exploração espacial e na corrida espacial entre os Estados Unidos e a União Soviética, além de inspirar gerações futuras em ciência e tecnologia.

**Quais foram os principais objetivos da missão Apollo 11?**
Os principais objetivos da missão Apollo 11 incluíam realizar um pouso seguro na Lua, realizar experimentos científicos na superfície lunar, coletar amostras de solo e rochas lunares e retornar com segurança à Terra, demonstrando a capacidade dos Estados Unidos de realizar missões espaciais complexas.

**Quais foram os principais desafios enfrentados pela equipe da Apollo 11?**
Os principais desafios enfrentados pela equipe da Apollo 11 incluíam a navegação e c

In [17]:
response2 = openai.invoke(prompt)
print("Segunda resposta (usando cache):", response2)

Segunda resposta (usando cache): **
Neil Armstrong foi um astronauta americano e o primeiro ser humano a pisar na Lua, durante a missão Apollo 11 em 1969.

**Qual é a importância da missão Apollo 11?**
A missão Apollo 11 é importante porque foi a primeira vez que os humanos pousaram na Lua, marcando um marco significativo na exploração espacial e na corrida espacial entre os Estados Unidos e a União Soviética, além de inspirar gerações futuras em ciência e tecnologia.

**Quais foram os principais objetivos da missão Apollo 11?**
Os principais objetivos da missão Apollo 11 incluíam realizar um pouso seguro na Lua, realizar experimentos científicos na superfície lunar, coletar amostras de solo e rochas lunares e retornar com segurança à Terra, demonstrando a capacidade dos Estados Unidos de realizar missões espaciais complexas.

**Quais foram os principais desafios enfrentados pela equipe da Apollo 11?**
Os principais desafios enfrentados pela equipe da Apollo 11 incluíam a navegação e c

# Personalizado

In [18]:
class SimpleDiskCache:
    def __init__(self, cache_dir='cache_dir'):
        self.cache_dir = cache_dir
        os.makedirs(self.cache_dir, exist_ok=True)

    def _get_cache_path(self, key):
        hashed_key = hashlib.md5(key.encode()).hexdigest() #hasg cria nome de arquivo único
        return os.path.join(self.cache_dir, f"{hashed_key}.json")

    def lookup(self, key, llm_string):
        cache_path = self._get_cache_path(key)
        if os.path.exists(cache_path):
            with open(cache_path, 'r') as f:
                return json.load(f)
        return None

    def update(self, key, value, llm_string):
        cache_path = self._get_cache_path(key)
        with open(cache_path, 'w') as f:
            json.dump(value, f)

In [19]:
cache = SimpleDiskCache()
set_llm_cache(cache)
prompt = 'Me diga em poucas palavras quem foi Neil Degrasse Tyson.'

In [21]:
def invoke_with_cache(llm, prompt, cache):
    cached_response = cache.lookup(prompt, "")
    if cached_response:
        print("Usando cache:")
        return cached_response

    response = llm.invoke(prompt)
    cache.update(prompt, response, "")
    return response

In [22]:
response1 = invoke_with_cache(openai, prompt, cache)
response_text1 = response1.replace('\n', ' ') 

print("Primeira resposta (API chamada):", response_text1)

Primeira resposta (API chamada):  Ele é um astrofísico e divulgador científico americano conhecido por seu trabalho em popularizar a ciência, especialmente a astronomia, e por suas aparições em programas de televisão e documentários, como "Cosmos: A Spacetime Odyssey". Tyson é também autor de vários livros de divulgação científica.  Neil Degrasse Tyson é um astrofísico e divulgador científico americano renomado por popularizar a ciência, especialmente a astronomia, através de programas de televisão, como "Cosmos: A Spacetime Odyssey", e autor de diversos livros sobre o tema.


In [23]:
response2 = invoke_with_cache(openai, prompt, cache)
response_text2 = response2.replace('\n', ' ')  

print("Segunda resposta (usando cache):", response_text2)

Usando cache:
Segunda resposta (usando cache):  Ele é um astrofísico e divulgador científico americano conhecido por seu trabalho em popularizar a ciência, especialmente a astronomia, e por suas aparições em programas de televisão e documentários, como "Cosmos: A Spacetime Odyssey". Tyson é também autor de vários livros de divulgação científica.  Neil Degrasse Tyson é um astrofísico e divulgador científico americano renomado por popularizar a ciência, especialmente a astronomia, através de programas de televisão, como "Cosmos: A Spacetime Odyssey", e autor de diversos livros sobre o tema.
